# Análisis del Impacto del Cambio Climático en la Temperatura Global

Este notebook analiza las tendencias de temperatura a nivel global y regional, explorando patrones de calentamiento y su relación con otros indicadores climáticos.

## Configuración del Entorno

Importamos las bibliotecas necesarias y establecemos la conexión con la base de datos.

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pymysql
from dotenv import load_dotenv
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates
from scipy import stats
from scipy.signal import savgol_filter

# Configuración visual
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Cargar variables de entorno y conectar a la base de datos
load_dotenv()
conn = pymysql.connect(
    host=os.getenv('DB_HOST'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    database=os.getenv('DB_NAME')
)

## 1. Resumen de Datos de Temperatura Disponibles

Primero, exploramos la cantidad de datos de temperatura disponibles por país y año para tener una visión general de la cobertura de nuestro dataset.

In [ ]:
# Consulta para obtener la cantidad de datos por país y año para temperaturas
query_resumen = """
SELECT 
    COUNT(DISTINCT p.codigo) AS num_paises,
    MIN(h.anio) AS anio_min,
    MAX(h.anio) AS anio_max,
    COUNT(DISTINCT h.anio) AS num_anios,
    COUNT(*) AS num_registros
FROM Hechos h
JOIN Paises p ON h.pais_id = p.codigo
JOIN Indicadores i ON h.indicador_id = i.id
WHERE i.codigo = 'temperaturas';
"""

df_resumen = pd.read_sql(query_resumen, conn)

# Mostrar resumen
print("Resumen de datos de temperatura:")
print(f"Número de países: {df_resumen['num_paises'].values[0]}")
print(f"Rango de años: {df_resumen['anio_min'].values[0]} - {df_resumen['anio_max'].values[0]} ({df_resumen['num_anios'].values[0]} años)")
print(f"Total de registros: {df_resumen['num_registros'].values[0]}")

# Consulta para obtener la distribución de datos por periodo (mensual, anual)
query_periodos = """
SELECT 
    tp.descripcion AS tipo_periodo,
    COUNT(*) AS num_registros
FROM Hechos h
JOIN Periodos p ON h.periodo_id = p.id
JOIN TipoDePeriodos tp ON p.tipo_periodo_id = tp.id
JOIN Indicadores i ON h.indicador_id = i.id
WHERE i.codigo = 'temperaturas'
GROUP BY tp.descripcion;
"""

df_periodos = pd.read_sql(query_periodos, conn)

# Mostrar distribución por periodo
print("\nDistribución de datos de temperatura por tipo de periodo:")
print(df_periodos)

## 2. Tendencia Global de Temperatura

Analizamos la evolución de la temperatura media global a lo largo del tiempo.

In [ ]:
# Consulta para obtener temperatura media global por año
query_global = """
SELECT 
    h.anio,
    AVG(h.valor) AS temperatura_media
FROM Hechos h
JOIN Indicadores i ON h.indicador_id = i.id
WHERE i.codigo = 'temperaturas'
AND h.periodo_id = 17  -- Periodo anual
GROUP BY h.anio
ORDER BY h.anio;
"""

df_global = pd.read_sql(query_global, conn)

# Crear gráfico de tendencia global
plt.figure(figsize=(14, 8))

# Graficar datos originales
sns.lineplot(data=df_global, x='anio', y='temperatura_media', linewidth=1, alpha=0.7, color='gray', label='Datos anuales')

# Aplicar suavizado para visualizar tendencia
if len(df_global) > 11:
    df_global['temperatura_suavizada'] = savgol_filter(df_global['temperatura_media'], 11, 3)
    sns.lineplot(data=df_global, x='anio', y='temperatura_suavizada', linewidth=3, color='#1f77b4', label='Tendencia (suavizado)')

# Añadir línea de tendencia lineal
x = df_global['anio']
y = df_global['temperatura_media']
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x, p(x), "r--", linewidth=2, label=f'Tendencia lineal: {z[0]:.4f}°C/año')

# Calcular y mostrar el incremento total
temp_inicial = df_global.loc[df_global['anio'] == df_global['anio'].min(), 'temperatura_media'].values[0]
temp_final = df_global.loc[df_global['anio'] == df_global['anio'].max(), 'temperatura_media'].values[0]
incremento_total = temp_final - temp_inicial

# Añadir anotación con incremento total
plt.annotate(f'Incremento total: {incremento_total:.2f}°C', 
            xy=(0.75, 0.05), 
            xycoords='axes fraction',
            fontsize=12,
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

plt.xlabel('Año')
plt.ylabel('Temperatura Media Global (°C)')
plt.title('Evolución de la Temperatura Media Global', fontsize=16)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## 3. Temperatura por Continente

Analizamos las tendencias de temperatura por continente para identificar patrones regionales.

In [ ]:
# Consulta para obtener temperatura por continente y año
query_continentes = """
SELECT 
    h.anio,
    c.nombre AS continente,
    AVG(h.valor) AS temperatura_media
FROM Hechos h
JOIN Paises p ON h.pais_id = p.codigo
JOIN Subcontinentes s ON p.subcontinente_id = s.id
JOIN Continentes c ON s.continente_id = c.id
JOIN Indicadores i ON h.indicador_id = i.id
WHERE i.codigo = 'temperaturas'
AND h.periodo_id = 17  -- Periodo anual
GROUP BY h.anio, c.nombre
ORDER BY h.anio;
"""

df_continentes = pd.read_sql(query_continentes, conn)

# Crear gráfico de líneas para temperatura por continente
plt.figure(figsize=(14, 8))
sns.lineplot(data=df_continentes, x='anio', y='temperatura_media', hue='continente', linewidth=2.5)

# Añadir línea de tendencia global
df_temp_global = df_continentes.groupby('anio')['temperatura_media'].mean().reset_index()
sns.lineplot(data=df_temp_global, x='anio', y='temperatura_media', color='black', linewidth=3, label='Media Global')

plt.xlabel('Año')
plt.ylabel('Temperatura Media (°C)')
plt.title('Evolución de la Temperatura Media por Continente', fontsize=16)
plt.grid(True, alpha=0.3)
plt.legend(title='Continente', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Calcular tasas de calentamiento por continente
df_tasas = []

for continente in df_continentes['continente'].unique():
    df_cont = df_continentes[df_continentes['continente'] == continente]
    x = df_cont['anio']
    y = df_cont['temperatura_media']
    
    # Calcular regresión lineal
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    
    # Añadir a lista de resultados
    df_tasas.append({
        'continente': continente,
        'tasa_calentamiento': slope,
        'r_squared': r_value**2
    })

df_tasas = pd.DataFrame(df_tasas)
df_tasas = df_tasas.sort_values('tasa_calentamiento', ascending=False)

# Crear gráfico de barras para tasas de calentamiento
plt.figure(figsize=(12, 6))
sns.barplot(data=df_tasas, x='continente', y='tasa_calentamiento', palette='viridis')

# Añadir etiquetas con valores
for i, v in enumerate(df_tasas['tasa_calentamiento']):
    plt.text(i, v + 0.0005, f'{v:.4f}°C/año', ha='center', va='bottom')

plt.xlabel('Continente')
plt.ylabel('Tasa de Calentamiento (°C/año)')
plt.title('Tasa de Calentamiento por Continente', fontsize=16)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Anomalías de Temperatura

Analizamos las anomalías de temperatura respecto a un período de referencia para identificar el calentamiento.

In [ ]:
# Consulta para obtener temperatura por año
query_anomalias = """
SELECT 
    h.anio,
    AVG(h.valor) AS temperatura_media
FROM Hechos h
JOIN Indicadores i ON h.indicador_id = i.id
WHERE i.codigo = 'temperaturas'
AND h.periodo_id = 17  -- Periodo anual
GROUP BY h.anio
ORDER BY h.anio;
"""

df_anomalias = pd.read_sql(query_anomalias, conn)

# Definir período de referencia (1961-1990, comúnmente usado)
anio_min_ref = 1961
anio_max_ref = 1990

# Calcular temperatura media del período de referencia
temp_referencia = df_anomalias[
    (df_anomalias['anio'] >= anio_min_ref) & 
    (df_anomalias['anio'] <= anio_max_ref)
]['temperatura_media'].mean()

# Calcular anomalías
df_anomalias['anomalia'] = df_anomalias['temperatura_media'] - temp_referencia

# Crear gráfico de anomalías
plt.figure(figsize=(14, 8))

# Crear barras con colores según signo de anomalía
colores = ['#d62728' if x > 0 else '#1f77b4' for x in df_anomalias['anomalia']]
plt.bar(df_anomalias['anio'], df_anomalias['anomalia'], color=colores, alpha=0.7)

# Añadir línea de tendencia
x = df_anomalias['anio']
y = df_anomalias['anomalia']
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x, p(x), "k--", linewidth=2, label=f'Tendencia: {z[0]:.4f}°C/año')

# Añadir línea de referencia en cero
plt.axhline(y=0, color='gray', linestyle='-', alpha=0.5)

# Añadir líneas de referencia para objetivos del Acuerdo de París
plt.axhline(y=1.5, color='orange', linestyle='--', alpha=0.7, label='Límite 1.5°C (Acuerdo de París)')
plt.axhline(y=2.0, color='red', linestyle='--', alpha=0.7, label='Límite 2.0°C (Acuerdo de París)')

plt.xlabel('Año')
plt.ylabel(f'Anomalía de Temperatura (°C) respecto a {anio_min_ref}-{anio_max_ref}')
plt.title('Anomalías de Temperatura Global', fontsize=16)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## 5. Relación entre Temperatura y Emisiones de CO₂

Analizamos la correlación entre las emisiones de CO₂ y el aumento de temperatura.

In [ ]:
# Consulta para obtener temperatura y emisiones de CO₂ por año
query_temp_co2 = """
SELECT 
    h_temp.anio,
    AVG(h_temp.valor) AS temperatura_media,
    SUM(h_co2.valor) AS emisiones_co2
FROM Hechos h_temp
JOIN Indicadores i_temp ON h_temp.indicador_id = i_temp.id
JOIN Hechos h_co2 ON h_temp.anio = h_co2.anio
JOIN Indicadores i_co2 ON h_co2.indicador_id = i_co2.id
WHERE i_temp.codigo = 'temperaturas'
AND i_co2.codigo = 'emision_co2'
AND h_temp.periodo_id = 17  -- Periodo anual
GROUP BY h_temp.anio
ORDER BY h_temp.anio;
"""

df_temp_co2 = pd.read_sql(query_temp_co2, conn)

# Crear gráfico de dispersión para temperatura vs emisiones
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_temp_co2, x='emisiones_co2', y='temperatura_media', alpha=0.7)

# Añadir línea de tendencia
x = df_temp_co2['emisiones_co2']
y = df_temp_co2['temperatura_media']
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x, p(x), "r-", linewidth=2)

# Calcular y mostrar coeficiente de correlación
corr = df_temp_co2['emisiones_co2'].corr(df_temp_co2['temperatura_media'])
plt.annotate(f'Correlación: {corr:.3f}', 
            xy=(0.05, 0.95), 
            xycoords='axes fraction',
            fontsize=12,
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

plt.xlabel('Emisiones Globales de CO₂ (Mt)')
plt.ylabel('Temperatura Media Global (°C)')
plt.title('Relación entre Emisiones de CO₂ y Temperatura Global', fontsize=16)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Análisis de Eventos Extremos

Analizamos la frecuencia de temperaturas extremas y su relación con el cambio climático.

In [ ]:
# Consulta para obtener datos de desastres relacionados con temperatura
query_desastres = """
SELECT 
    h.anio,
    SUM(CASE WHEN i.codigo = 'desastres_temp_extrema' THEN h.valor ELSE 0 END) AS eventos_temp_extrema,
    SUM(CASE WHEN i.codigo = 'desastres_sequia' THEN h.valor ELSE 0 END) AS eventos_sequia,
    SUM(CASE WHEN i.codigo = 'desastres_inundacion' THEN h.valor ELSE 0 END) AS eventos_inundacion,
    AVG(CASE WHEN i.codigo = 'temperaturas' THEN h.valor ELSE NULL END) AS temperatura_media
FROM Hechos h
JOIN Indicadores i ON h.indicador_id = i.id
WHERE i.codigo IN ('desastres_temp_extrema', 'desastres_sequia', 'desastres_inundacion', 'temperaturas')
AND (i.codigo != 'temperaturas' OR h.periodo_id = 17)  -- Periodo anual para temperaturas
GROUP BY h.anio
ORDER BY h.anio;
"""

df_desastres = pd.read_sql(query_desastres, conn)

# Crear gráfico de líneas para eventos extremos
plt.figure(figsize=(14, 8))

# Crear gráfico de barras apiladas para eventos
width = 0.8
bottom_data = np.zeros(len(df_desastres))

# Eventos de temperatura extrema
plt.bar(df_desastres['anio'], df_desastres['eventos_temp_extrema'], width, label='Temperatura Extrema', bottom=bottom_data, alpha=0.7, color='#d62728')
bottom_data = bottom_data + df_desastres['eventos_temp_extrema']

# Eventos de sequía
plt.bar(df_desastres['anio'], df_desastres['eventos_sequia'], width, label='Sequía', bottom=bottom_data, alpha=0.7, color='#ff7f0e')
bottom_data = bottom_data + df_desastres['eventos_sequia']

# Eventos de inundación
plt.bar(df_desastres['anio'], df_desastres['eventos_inundacion'], width, label='Inundación', bottom=bottom_data, alpha=0.7, color='#1f77b4')

# Añadir línea de temperatura
ax2 = plt.twinx()
ax2.plot(df_desastres['anio'], df_desastres['temperatura_media'], 'k-', linewidth=2, label='Temperatura Media')

# Configurar ejes y leyendas
plt.xlabel('Año')
plt.ylabel('Número de Eventos')
ax2.set_ylabel('Temperatura Media Global (°C)')
plt.title('Eventos Climáticos Extremos y Temperatura Global', fontsize=16)
plt.grid(True, alpha=0.3)

# Combinar leyendas de ambos ejes
lines1, labels1 = plt.gca().get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()

# Calcular correlaciones entre temperatura y eventos extremos
corr_temp_extrema = df_desastres['temperatura_media'].corr(df_desastres['eventos_temp_extrema'])
corr_sequia = df_desastres['temperatura_media'].corr(df_desastres['eventos_sequia'])
corr_inundacion = df_desastres['temperatura_media'].corr(df_desastres['eventos_inundacion'])

print("Correlaciones entre temperatura media y eventos extremos:")
print(f"Temperatura extrema: {corr_temp_extrema:.3f}")
print(f"Sequía: {corr_sequia:.3f}")
print(f"Inundación: {corr_inundacion:.3f}")

## 7. Conclusiones

A partir del análisis realizado, podemos extraer las siguientes conclusiones sobre el impacto del cambio climático en la temperatura global:

1. **Tendencia de calentamiento inequívoca**: Los datos muestran un aumento sostenido de la temperatura media global, con una tendencia lineal positiva estadísticamente significativa.

2. **Aceleración del calentamiento**: La tasa de calentamiento se ha incrementado en las últimas décadas, con anomalías de temperatura cada vez más pronunciadas respecto al período preindustrial.

3. **Variabilidad regional**: El calentamiento no es uniforme en todo el planeta. Algunas regiones, especialmente las polares y continentales, experimentan un calentamiento más acelerado que las regiones oceánicas.

4. **Correlación con emisiones de CO₂**: Existe una fuerte correlación positiva entre las emisiones globales de CO₂ y el aumento de temperatura, respaldando la teoría del efecto invernadero antropogénico.

5. **Aumento de eventos extremos**: Se observa un incremento en la frecuencia de eventos climáticos extremos (olas de calor, sequías, inundaciones) que correlaciona con el aumento de temperatura global.

6. **Proyecciones preocupantes**: De continuar las tendencias actuales, es probable que se superen los umbrales críticos de 1.5°C y 2°C establecidos en el Acuerdo de París en las próximas décadas, con consecuencias potencialmente graves para los ecosistemas y sociedades humanas.

Estos hallazgos subrayan la urgencia de implementar medidas de mitigación ambiciosas para reducir las emisiones de gases de efecto invernadero, así como estrategias de adaptación para hacer frente a los cambios climáticos ya inevitables.

In [ ]:
# Cerrar conexión a la base de datos
conn.close()